In [ ]:
!pip install minio

In [ ]:
param_s3_server = "scruffy.lab.uvalight.net:9000"
param_s3_access_key = "vl-biodt-demo"
# Add the secret key to run the notebook
param_s3_secret_key = ""

param_s3_bucket_input = "naa-vre-public"
param_s3_prefix_input = "vl-biodt-demo/"

param_s3_bucket_output = "naa-vre-user-data"
param_s3_prefix_output = "vl-biodt-demo/"

conf_data_dir = '/tmp/data'

param_grid_size_para = 25

# Zonalfilter
param_zone_field_para= "id"

In [ ]:
# cimpal-get-files

import os
from minio import Minio


minio_client = Minio(param_s3_server, access_key=param_s3_access_key, secret_key=param_s3_secret_key, secure=True)

for item in minio_client.list_objects(param_s3_bucket_input, prefix=f"{param_s3_prefix_input}", recursive=True):
    target_file = f"{conf_data_dir}/input/{item.object_name.removeprefix(param_s3_prefix_input)}"
    if not os.path.exists(target_file):
        print("Downloading", item.object_name)
        minio_client.fget_object(param_s3_bucket_input, item.object_name, target_file)

occ_taxa = f"{conf_data_dir}/input/Cimpal_resources"
biotope_shp_path_file = f"{conf_data_dir}/input/Cimpal_resources"
weight_file = f"{conf_data_dir}/input/Cimpal_resources/weight_wp.csv"
pathway_file = f"{conf_data_dir}/input/Cimpal_resources/CIMPAL_paths.csv"
zones_file = f"{conf_data_dir}/input/zones"

In [ ]:
# cimpal-calculator
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - occ_taxa: String
#    - biotope_shp_path_file: String
#    - weight_file: String
#    - pathway_file: String
#   outputs:
#    - out_path: String
# ...

import os

# inputs
occ_and_taxa_path = occ_taxa
biotope_shp_path = biotope_shp_path_file
weights_path = weight_file
pathways_path = pathway_file

# outputs
os.makedirs(f"{conf_data_dir}/output/Cimpal")
out_path = f"{conf_data_dir}/output/Cimpal"



import glob, subprocess, pandas
import os, re, csv

from osgeo import ogr, osr

from osgeo import gdal

import pyproj
from shapely.geometry import MultiPolygon, Polygon, Point
from shapely.ops import transform
from functools import partial
from pathlib import Path

import argparse

# parser = argparse.ArgumentParser(description="Biotope Cimpal Calculator")

# parser.add_argument(
#     "-ot",
#     "--occ_and_taxa",
#     type=str,
#     help="Occurrences And Alien Taxonomy Dir",
#     required=True,
# )
# parser.add_argument(
#     "-bshp", "--biotope_shp", type=str, help="Biotope's Shp Dir", required=True
# )
# parser.add_argument(
#     "-p", "--pathways", type=str, help="Pathways csv file", required=False
# )
# parser.add_argument("-w", "--weights", type=str, help="Weights csv file", required=True)
# parser.add_argument(
#     "-eea", "--useEEA", type=bool, help="Use EEA projection", required=True
# )
# parser.add_argument("-grid", "--grid_size", type=int, help="Grid Size", required=True)
# parser.add_argument("-o", "--out", type=str, help="outpath", required=True)

# args = parser.parse_args()
gbif_dir = Path(occ_and_taxa_path)
shp_dir = biotope_shp_path
shp_files = os.listdir(shp_dir)
impact_matrix = weights_path
grid_size = param_grid_size_para
nis_pathways_matrix_path_dir = pathways_path

# occurences = "/export/projects/LIFEWATCH/cimpal/metz/occur_RiaAvei_Pinheiro2019_LWformat_2/occur_RiaAvei_Pinheiro2019_LWformat_2.csv"
# alien_taxa_csv = (
#     "/export/projects/LIFEWATCH/cimpal/metz/Alien_taxa_List/" + "Alien_taxa_List.csv"
# )

# set the paths to the apps
otbbin = "/usr/local/otb/bin/"
outpath = out_path

shp_file = None

for file in shp_files:
    if file.endswith(".shp"):
        shp_file = os.path.join(shp_dir, file)
        break

if not shp_file:
    raise FileNotFoundError("`.shp` file not found in zip")

# get input file names from orchestrator

biotopes = str(shp_dir) + '/'
outpath = out_path + '/'
weight_matrix_template = weights_path
filter_matrix_csv = pathways_path
occurences = str(Path(gbif_dir, "occurrence.txt"))
alienTaxa = str(Path(gbif_dir, "alientaxa.txt"))

print("=================================")
print(f"biotopes {biotopes}")
print(f"outpath {outpath}")
print(f"weight_matrix_template {weight_matrix_template}")
print(f"filter_matrix_csv {filter_matrix_csv}")
print(f"occurences {occurences}")
print(f"alienTaxa {alienTaxa}")
print("=================================")



# # predefined variables
zone_field = 'id_habitat'
field_name_oc = "scientificName"
useEEA = True
LAEA=True
grid_size = int(param_grid_size_para)
field_name_radius = "dispersionRadius"
radius_default = 100
field_name = "Habitat"

## A. Convert all data about habitat into raster layers on a single grid
## A.1 Check that all data are in the same coordinate system and compute the maximum extent
##
extent_tot = [
    100000000,
    -100000000,
    100000000,
    -100000000,
]  # initialise extent: xmin, xmax, ymin, ymax

if useEEA:
    shp_candidates = glob.glob(biotopes + "[sS]ingle_*.shp") + glob.glob(
        biotopes + "[Ll]ist_*.shp"
    )
    print(len(shp_candidates))
    for shp in [x for x in shp_candidates if not "LAEA" in x]:
        # reproject the dataset
        print("reprojecting " + shp)
        outshp = shp[:-4] + "_LAEA.shp"
        subprocess.call(["ogr2ogr", "-t_srs", "EPSG:3035", outshp, shp])
        driver = ogr.GetDriverByName("ESRI Shapefile")
        dataSource = driver.Open(outshp, 0)  # 0 means read-only. 1 means writeable.

        # Check to see if shapefile is found.
        if dataSource is None:
            print("Could not open %s" % (shp))
        else:
            inLayer = dataSource.GetLayer()
            extent = inLayer.GetExtent()
            if extent[0] < extent_tot[0]:
                extent_tot[0] = extent[0]
            if extent[1] > extent_tot[1]:
                extent_tot[1] = extent[1]
            if extent[2] < extent_tot[2]:
                extent_tot[2] = extent[2]
            if extent[3] > extent_tot[3]:
                extent_tot[3] = extent[3]
    grid_srs = osr.SpatialReference()
    grid_srs.ImportFromEPSG(3035)
else:
    srs_list = []
    refdict = {}
    for shp in glob.glob(biotopes + "[Ss]ingle_*.shp")  + glob.glob(
        biotopes + "[Ll]ist_*.shp"
    ):
        driver = ogr.GetDriverByName("ESRI Shapefile")
        dataSource = driver.Open(shp, 0)  # 0 means read-only. 1 means writeable.
        # Check to see if shapefile is found.
        if dataSource is None:
            print("Could not open %s" % (shp))
        else:
            inLayer = dataSource.GetLayer()
            refdict[shp] = inLayer.GetSpatialRef()
            srs_list.append(inLayer.GetSpatialRef())
            extent = inLayer.GetExtent()
            if extent[0] < extent_tot[0]:
                extent_tot[0] = extent[0]
            if extent[1] > extent_tot[1]:
                extent_tot[1] = extent[1]
            if extent[2] < extent_tot[2]:
                extent_tot[2] = extent[2]
            if extent[3] > extent_tot[3]:
                extent_tot[3] = extent[3]
    grid_srs = srs_list[0]
    for my_srs in srs_list:
        if my_srs.IsSame(grid_srs):
            print("OK")
        else:
            print("all data must be in the same coordinate system")
            for layer_ref in refdict.items():
                print(
                    layer_ref[0]
                    + " has coordinate system "
                    + layer_ref[1].ExportToPrettyWkt()
                )
            sys.exit()

print("Full extent based on input files")
print(extent_tot)
x_min, x_max, y_min, y_max = extent_tot


x_min = ((x_min // grid_size) - 1) * grid_size
y_min = ((y_min // grid_size) - 1) * grid_size
x_max = ((x_max // grid_size) + 1) * grid_size
y_max = ((y_max // grid_size) + 1) * grid_size


for shp in glob.glob(biotopes + "[Ll]ist_*.shp"):
    dataSource = driver.Open(shp, 0)
    layer = dataSource.GetLayer()
    values_list = []
    for feature in layer:
        newval = feature.GetField(field_name)
        values_list.append(newval)
    layer.ResetReading()
    for fieldval in list(set(values_list)):
        layer.SetAttributeFilter("{0} = '{1}'".format(field_name, fieldval))
        # Create the output LayerS
        filename = re.sub("[^a-zA-Z0-9\n\.]", "_", fieldval).lower()
        print(filename)
        outShapefile = os.path.join(
            os.path.split(shp)[0], "single_{}.shp".format(filename)
        )
        outDriver = ogr.GetDriverByName("ESRI Shapefile")
        # Remove output shapefile if it already exists
        if os.path.exists(outShapefile):
            outDriver.DeleteDataSource(outShapefile)
        # Create the output shapefile
        outDataSource = outDriver.CreateDataSource(outShapefile)
        out_lyr_name = os.path.splitext(os.path.split(outShapefile)[1])[0]
        outLayer = outDataSource.CreateLayer(
            out_lyr_name, grid_srs, geom_type=ogr.wkbMultiPolygon
        )
        # Get the output Layer's Feature Definition
        outLayerDefn = outLayer.GetLayerDefn()
        # Add features to the ouput Layer
        for inFeature in layer:
            # Create output Feature
            outFeature = ogr.Feature(outLayerDefn)
            # Set geometry as centroid
            geom = inFeature.GetGeometryRef()
            outFeature.SetGeometry(geom.Clone())
            # Add new feature to output Layer
            outLayer.CreateFeature(outFeature)
        # Close DataSources
        layer.ResetReading()
        outDataSource.Destroy()

if LAEA:
    biotopes_proj = biotopes + "[Ss]ingle*_LAEA.shp"
else:
    biotopes_proj = biotopes + "[Ss]ingle*.shp"
for shp in glob.glob(biotopes_proj):
    
    # Define grid_size and NoData value of new raster
    NoData_value = 0
    # Filename of the raster Tiff that will be created
    norm_name = re.sub("[^a-zA-Z0-9\n\.]", "_", os.path.split(shp)[1])
    out_name = norm_name[:-4].replace("single_", "h_") + ".tif"
    raster_fn = os.path.join(os.path.split(shp)[0], out_name)
    # Open the data source and read in the extent
    source_ds = ogr.Open(shp)
    source_layer = source_ds.GetLayer()
    source_srs = source_layer.GetSpatialRef()
    # Create the destination data source
    x_res = int((x_max - x_min) / grid_size)
    y_res = int((y_max - y_min) / grid_size)
    target_ds = gdal.GetDriverByName("GTiff").Create(
        raster_fn, x_res, y_res, gdal.GDT_Byte
    )
    target_ds.SetGeoTransform((x_min, grid_size, 0, y_max, 0, -grid_size))
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(NoData_value)
    target_ds.SetProjection(source_srs.ExportToWkt())
    # Rasterize
    gdal.RasterizeLayer(
        target_ds, [1], source_layer, burn_values=[1], options=["ALL_TOUCHED=TRUE"]
    )

##
#### B create raster for each species based on an uncertainty radius
##
wgs84_globe = pyproj.Proj(proj="latlong", ellps="WGS84")

# function for a geodetic buffer around a point
def point_buff_geodetic(c1, c2, radius, in_proj, out_proj):
    # get the equidistant projection centered on the point
    _lon, _lat = pyproj.transform(in_proj, wgs84_globe, c1, c2)
    aeqd = pyproj.Proj(
        proj="aeqd", ellps="WGS84", datum="WGS84", lat_0=_lat, lon_0=_lon
    )
    # print(aeqd)
    # buffer in azimuthal equidistant projection and project it to its initial srs
    return sh_transform(
        partial(pyproj.transform, aeqd, out_proj), Point(0, 0).buffer(radius)
    )

def point_buff_geodetic_wgs(c1, c2, radius):
    # get the equidistant projection centered on the point
    aeqd = pyproj.Proj(proj="aeqd", ellps="WGS84", datum="WGS84", lat_0=c2, lon_0=c1)
    # print("internal longitude = "  + str(c1))
    # print(aeqd)
    # buffer in azimuthal equidistant projection and project it to its initial srs
    proj4str = "+proj=aeqd +lat_0=%s +lon_0=%s +x_0=0 +y_0=0" % (c2, c1)
    aeqd = pyproj.Proj(proj4str)  # azimuthal equidistant
    project = partial(pyproj.transform, aeqd, pyproj.Proj("EPSG:4326"), always_xy=True)
    return transform(project, Point(0, 0).buffer(radius))


###B.1 get the list of all species
print(grid_srs.ExportToWkt())
hab_proj = pyproj.Proj(grid_srs.ExportToWkt())
if ".shp" in occurences:
    dataSource = driver.Open(occurences, 0)
    sp_layer = dataSource.GetLayer()
    field_name_oc = "scientific"
    field_name_radius = "radius"
    sp_srs = sp_layer.GetSpatialRef()
    sp_proj = pyproj.Proj(sp_srs.ExportToWkt())
    species_list = []
    for feature in sp_layer:
        newval = feature.GetField(field_name_oc)
        if not newval in species_list:
            species_list.append(newval)
    sp_layer.ResetReading()
    print("there are " + str(len(species_list)) + " items")

    for fieldval in species_list:
        sp_layer.SetAttributeFilter("{0} = '{1}'".format(field_name_oc, fieldval))
        # Create the output LayerS
        filename = re.sub("[^a-zA-Z0-9\n\.]", "_", fieldval).lower()
        print(filename)
        outShapefile = os.path.join(
            os.path.split(occurences)[0], "spf_{}.shp".format(filename)
        )
        outDriver = ogr.GetDriverByName("ESRI Shapefile")
        # Remove output shapefile if it already exists
        if os.path.exists(outShapefile):
            outDriver.DeleteDataSource(outShapefile)
        # Create the output shapefile
        outDataSource = outDriver.CreateDataSource(outShapefile)
        out_lyr_name = os.path.splitext(os.path.split(outShapefile)[1])[0]
        outLayer = outDataSource.CreateLayer(
            out_lyr_name, grid_srs, geom_type=ogr.wkbMultiPolygon
        )
        # Get the output Layer's Feature Definition
        outLayerDefn = outLayer.GetLayerDefn()
        # print(outLayerDefn)
        # Add features to the ouput Layer
        for inFeature in sp_layer:
            # Create output Feature
            outFeature = ogr.Feature(outLayerDefn)
            # Set geometry as centroid
            geom = inFeature.GetGeometryRef()
            uncertainty_radius = inFeature.GetField(field_name_radius)
            geom_out = point_buff_geodetic(
                geom.GetX(), geom.GetY(), uncertainty_radius, sp_proj, hab_proj
            )
            poly = ogr.CreateGeometryFromWkt(geom_out.wkt)
            # print(poly)
            multipolygon = ogr.Geometry(ogr.wkbMultiPolygon)
            multipolygon.AddGeometry(poly)
            outFeature.SetGeometry(multipolygon)
            # Add new feature to output Layer
            outLayer.CreateFeature(outFeature)
            # inFeature.Destroy()
        # Close DataSources
        sp_layer.ResetReading()
        outDataSource.Destroy()
elif ".txt" in occurences:
    radius = 50
    print("csv input")
    # get a list of alien taxa for filtering
    alien_taxa = {}
    with open(alienTaxa, newline="") as csvfile_at:
        atl = csv.reader(csvfile_at, delimiter="\t", quotechar="|")
        i = 0
        for row in atl:
            if i == 0:
                fld_idx = row.index("scientificName")
                try:
                    radius_idx = row.index("dispersionRadius")
                except:
                    radius_idx = -1
                i += 1
            else:
                if radius_idx < 0:
                    alien_taxa[row[fld_idx]] = radius_default
                else:
                    alien_taxa[row[fld_idx]] = float(row[radius_idx])
    print("there are " + str(len(alien_taxa)) + " items")
    df_occs = pandas.read_csv(
        occurences,
        delimiter="\t",
        usecols=[
            field_name_oc,
            "decimalLatitude",#-"NCoord",
            "decimalLongitude",#-"ECoord",
        ],
    )
    sp_out_srs = osr.SpatialReference()
    sp_out_srs.ImportFromEPSG(4326)
    for alien_sp in alien_taxa:
        # get the info from the dict
        print(alien_sp + " " + str(round(alien_taxa[alien_sp], 0)))
        dis_radius = alien_taxa[alien_sp]
        df_temp = df_occs.loc[df_occs[field_name_oc] == alien_sp]
        filename = re.sub("[^a-zA-Z0-9\n\.]", "_", alien_sp).lower()
        outShapefile_csv = os.path.join(
            os.path.split(occurences)[0], "sp_{}.shp".format(filename)
        )
        outDriver = ogr.GetDriverByName("ESRI Shapefile")
        # Remove output shapefile if it already exists
        if os.path.exists(outShapefile_csv):
            outDriver.DeleteDataSource(outShapefile_csv)
        # Create the output shapefile
        outDataSource_csv = outDriver.CreateDataSource(outShapefile_csv)
        out_lyr_name_csv = os.path.splitext(os.path.split(outShapefile_csv)[1])[0]
        outLayer_csv = outDataSource_csv.CreateLayer(
            out_lyr_name_csv, sp_out_srs, geom_type=ogr.wkbMultiPolygon
        )
        # Get the output Layer's Feature Definition
        outLayerDefn_csv = outLayer_csv.GetLayerDefn()
        # print(outLayerDefn)
        # Add features to the ouput Layer
        j = 0
        for i, row in df_temp.iterrows():
            # Create output Feature
            outFeature_csv = ogr.Feature(outLayerDefn_csv)
            # Set geometry coordinates
            geom_out = point_buff_geodetic_wgs(
                #-float(row.NCoord), float(row.ECoord), dis_radius
                float(row.decimalLongitude) ,float(row.decimalLatitude),dis_radius
            )  # float(row.DispersionRadius))
            poly = ogr.CreateGeometryFromWkt(geom_out.wkt)
            # print(poly)
            multipolygon = ogr.Geometry(ogr.wkbMultiPolygon)
            multipolygon.AddGeometry(poly)
            outFeature_csv.SetGeometry(multipolygon)
            # Add new feature to output Layer
            outLayer_csv.CreateFeature(outFeature_csv)
            j += 1
            # inFeature.Destroy()
        # Close DataSources
        print(filename + " has" + str(j) + " features")
        outDataSource_csv.Destroy()
        grid_srs.AutoIdentifyEPSG()
        subprocess.call(
            [
                "ogr2ogr",
                "-t_srs",
                "EPSG:" + str(grid_srs.GetAuthorityCode(None)),
                outShapefile_csv[:-4] + "_loc.shp",
                outShapefile_csv,
            ]
        )
else:
    print("No valid imput for species occurences")


## C.2 rasterize the species occurrences (one file per species)

for shp in glob.glob(os.path.join(os.path.split(occurences)[0], "sp_*loc.shp")):

    # Define NoData value of new raster
    NoData_value = 0

    # Filename of the raster Tiff that will be created
    raster_fn = shp[:-4].replace("sp_", "s_") + ".tif"
    print(raster_fn + " raster processing")
    # Open the data source and read in the extent
    source_ds = ogr.Open(shp)
    source_layer = source_ds.GetLayer()
    source_srs = source_layer.GetSpatialRef()

    # Create the destination data source
    x_res = int((x_max - x_min) / grid_size)
    y_res = int((y_max - y_min) / grid_size)
    target_ds = gdal.GetDriverByName("GTiff").Create(
        raster_fn, x_res, y_res, gdal.GDT_Byte
    )
    target_ds.SetGeoTransform((x_min, grid_size, 0, y_max, 0, -grid_size))
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(NoData_value)
    target_ds.SetProjection(source_srs.ExportToWkt())
    # Rasterize
    gdal.RasterizeLayer(
        target_ds, [1], source_layer, burn_values=[1], options=["ALL_TOUCHED=TRUE"]
    )


# C compute CIMPAL value based on habitat and occurence files
##

for weight_matrix_csv in [weight_matrix_template]:
    w = os.path.split(weight_matrix_csv)[1][:-4]
    print(w)
    with open(weight_matrix_csv, newline="") as csvfile:
        w_matrix = csv.reader(csvfile, delimiter="\t", quotechar="|")
        i = 0
        for row in w_matrix:
            print(f"{row}")
            if i == 0:
                habitat_names = [
                    re.sub("[^a-zA-Z0-9\n\.]", "_", x).lower() for x in row
                ]
                i += 1
            else:
                sp_name = re.sub("[^a-zA-Z0-9\n\.]", "_", row[0]).lower()
                sp_file = os.path.join(
                    os.path.split(occurences)[0], "s_{}_loc.tif".format(sp_name)
                )
                if os.path.exists(sp_file):
                    j = 0
                    k = 1
                    expression = []
                    file_list = [sp_file]
                    for habitat in row:
                        if j > 0:
                            if LAEA:
                                hab_file = os.path.join(
                                    biotopes, "h_{}_LAEA.tif".format(habitat_names[j])
                                    )
                            else:
                                hab_file = os.path.join(
                                    biotopes, "h_{}.tif".format(habitat_names[j])
                                    )
                            if os.path.exists(hab_file):
                                if float(habitat) > 0:
                                    k += 1
                                    expression.append(
                                        "im1b1*im{0}b1*{1}".format(k, habitat)
                                    )
                                    file_list.append(hab_file)
                            else:
                                print(
                                    "WARNING: no file for {}".format(habitat_names[j])
                                )
                                expression.append("0") #use zero in case a file is not found to avoid NoData rasters
                            j += 1
                        else:
                            j += 1
                    if len(file_list) > 0:
                        if len(expression) == 0:
                            expression = ["0"]
                        outfile = os.path.join(
                            outpath,
                            "impact_{0}_{1}.tif{2}".format(
                                sp_name,
                                w,
                                "?&gdal:co:COMPRESS=LZW&gdal:co:TILED=YES&gdal:co:BIGTIFF=YES",
                            ),
                        )
                        print("+".join(expression))
                        subprocess.call(
                            [
                                otbbin + "otbcli_BandMath",
                                "-out",
                                outfile,
                                "-exp",
                                "+".join(expression),
                                "-il",
                            ]
                            + file_list
                        )
                else:
                    print("WARNING: no file for {}".format(sp_name))
    if filter_matrix_csv and os.path.exists(filter_matrix_csv):
        with open(filter_matrix_csv, newline="") as csvfile_f:
            f_matrix = csv.reader(csvfile_f, delimiter="\t", quotechar="|")
            i = 0
            for row in f_matrix:
                if i == 0:
                    sp_names = [re.sub("[^a-zA-Z0-9\n\.]", "_", x).lower() for x in row]
                    i += 1
                    print("number of species " + str(len(sp_names)))
                else:
                    j = 0
                    k = 0
                    expression = []
                    sp_file_list = []
                    for sp in row:
                        if j > 0:
                            impact_file = os.path.join(
                                outpath, "impact_{0}_{1}.tif".format(sp_names[j], w)
                            )
                            if os.path.exists(impact_file):
                                if float(sp) > 0:
                                    k += 1
                                    expression.append("im{0}b1".format(k))
                                    sp_file_list.append(impact_file)
                            else:
                                print(j)
                                print("WARNING: no file for {}".format(sp_names[j]))
                                expression.append("0") # add zero in case of missing file to avoid nodata in further calculations
                            j += 1
                        else:
                            j += 1
                    if len(sp_file_list) > 0:
                        outfile = os.path.join(
                            outpath,
                            "cimpal_{0}_{1}.tif{2}".format(
                                row[0],
                                w,
                                "?&gdal:co:COMPRESS=LZW&gdal:co:TILED=YES&gdal:co:BIGTIFF=YES",
                            ),
                        )
                        subprocess.call(
                            [
                                otbbin + "otbcli_BandMath",
                                "-out",
                                outfile,
                                "-exp",
                                "+".join(expression),
                                "-il",
                            ]
                            + sp_file_list
                        )
                    else:
                        print("WARNING: no file for pathway {row[0]} and matrix {w}")
    else:
        k = 1
        expression = []
        sp_file_list = []
        for impact_file in glob.glob(
            os.path.join(outpath, "impact_*_{0}.tif".format(w))
        ):
            expression.append("im{0}b1".format(k))
            sp_file_list.append(impact_file)
            k += 1
        if len(sp_file_list) > 0:
            outfile = os.path.join(
                outpath,
                "cimpal_all_{0}.tif{1}".format(
                    w, "?&gdal:co:COMPRESS=LZW&gdal:co:TILED=YES&gdal:co:BIGTIFF=YES"
                ),
            )
            subprocess.call(
                [
                    otbbin + "otbcli_BandMath",
                    "-out",
                    outfile,
                    "-exp",
                    "+".join(expression),
                    "-il",
                ]
                + sp_file_list
            )
        else:
            print(f"WARNING: No impact files matrix {w}")
grid_srs_out = Path(outpath, "grid_srs_toWkt.txt")
f = open(grid_srs_out, mode="w+")
f.write(grid_srs.ExportToWkt())
f.close()


In [ ]:
# cimpal-zonal-filter

gris_srs_to_Wkt_path = out_path + "/grid_srs_toWkt.txt"
maps_path = out_path

shps_path = zones_file

import os

def create_directory_if_not_exists(path):
    if not os.path.exists(path):
        try:
            os.makedirs(path)
            print("Directory created successfully at:", path)
        except OSError as e:
            print("Error creating directory:", e)
    else:
        print("Directory already exists at:", path)



create_directory_if_not_exists(f"{conf_data_dir}/output/Zonal_out")
out_path2 = f"{conf_data_dir}/output/Zonal_out"


species_para = True


# -*- coding: utf-8 -*-
##"""
##LIFEWATCH_ZONAL_STATISTICS - Copyright (C) <2021>  Université catholique de Louvain (UCLouvain), Belgique,
##
##
##
##List of the contributors to the development of LIFEWATCH_CIMPAL: see LICENSE file.
##Description and complete License: see LICENSE file.
##
##This program (LIFEWATCH_ZONAL_STATISTICS) is free software:
##you can redistribute it and/or modify it under the terms of the
##GNU General Public License as published by the Free Software
##Foundation, either version 3 of the License, or (at your option)
##any later version.
##This program is distributed in the hope that it will be useful,
##but WITHOUT ANY WARRANTY; without even the implied warranty of
##MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
##GNU General Public License for more details.
##You should have received a copy of the GNU General Public License
##along with this program (see COPYING file).  If not,
##see <http://www.gnu.org/licenses/>.
##"""

##
import subprocess, glob
import os

from osgeo import gdal

    
from pathlib import Path

import argparse

os.environ['PROJ_LIB'] = '/usr/local/otb/share/proj/'

# parser = argparse.ArgumentParser(description="Biotope Cimpal Calculator")

# parser.add_argument("-s", "--shps", type=str, help="Shps Dir", required=True)
# parser.add_argument("-m", "--maps", type=str, help="Maps Dir", required=True)
# parser.add_argument("-zf", "--zone_field", type=str, help="Zone field id", required=True)
# parser.add_argument("-srs", "--gris_srs_to_Wkt", type=str, help="grid srs txt", required=True)
# parser.add_argument("-o", "--out", type=str, help="Out dir", required=True)
# parser.add_argument("-sp", "--species", type=bool, help="Run Zonal on species files as well", default=True)

# args = parser.parse_args()
shp_dir = shps_path
shp_files = os.listdir(shp_dir)

maps_dir = maps_path

grid_srs_to_Wkt = gris_srs_to_Wkt_path

if Path(grid_srs_to_Wkt).exists():
    with open(grid_srs_to_Wkt, "r") as f:
        grid_srs_to_Wkt = f.read()

out = out_path2

shp_file = None

for file in shp_files:
    if file.endswith(".shp"):
        shp_file = os.path.join(shp_dir, file)
        break

# get input file names from orchestrator
shp_zone = shp_file

zone_field = param_zone_field_para
zonal = Path("/usr/local/lw_apps/lwZonalStatistics")

# Filename of the raster Tiff that will be created
raster_z = shp_zone[:-4] + ".tif"

tif_files = glob.glob(os.path.join(maps_dir, "*.tif"))

#get the characteristics of the file 
hDataset = gdal.Open(tif_files[0], gdal.GA_ReadOnly)
adfGeoTransform = hDataset.GetGeoTransform(can_return_null=True)

if adfGeoTransform is not None:
    dfGeoXUL = adfGeoTransform[0]
    dfGeoYUL = adfGeoTransform[3]
    dfGeoXLR = (
        adfGeoTransform[0] + adfGeoTransform[1] * hDataset.RasterXSize + adfGeoTransform[2] * hDataset.RasterYSize
    )
    dfGeoYLR = (
        adfGeoTransform[3] + adfGeoTransform[4] * hDataset.RasterXSize + adfGeoTransform[5] * hDataset.RasterYSize
    )
    xres = str(abs(adfGeoTransform[1]))
    yres = str(abs(adfGeoTransform[5]))

# Project to the same coordinate system than the grid
outshp = shp_zone[:-4] + "_gridSRS.shp"

subprocess.call(["ogr2ogr", "-progress", "-t_srs", grid_srs_to_Wkt, outshp, shp_zone])
subprocess.call(
    [
        "gdal_rasterize",
        "-a",
        zone_field,
        "-ot",
        "uint16",
        "-co",
        "BIGTIFF=YES",
        "-co",
        "TILED=YES",
        "-co",
        "COMPRESS=DEFLATE",
        "-te",
        str(dfGeoXUL),
        str(dfGeoYLR),
        str(dfGeoXLR),
        str(dfGeoYUL),
        "-tr",
        str(xres),
        str(yres),
        outshp,
        raster_z,
    ]
)

if not species_para:
    tif_files =[f for f in tif_files if 'cimpal_' in f]

for file in tif_files:
    is_cimpal = True
    print(f"Running for: {file}")
    
    tabular_out = Path(
        out, os.path.split(file)[1][:-4] + "_stat.csv",
    )
    
    print(f"{str(tabular_out)}")

    subprocess.call([str(zonal), raster_z, file, tabular_out])

In [ ]:
# cimpal-upload-results

import base64
import os
import uuid
from minio import Minio

minio_client = Minio(param_s3_server, access_key=param_s3_access_key, secret_key=param_s3_secret_key, secure=True)

unique_folder = str(uuid.uuid4())

for filename in os.listdir(out_path2):
    file_path = f"{out_path2}/{filename}"
    dir_basename = os.path.basename(out_path2)
    user_prefix = param_s3_prefix_output[:-1]
    object_name = f"{user_prefix}/{unique_folder}/{dir_basename}/{filename}"
    print("Uploading", file_path, "->", object_name)
    minio_client.fput_object(param_s3_bucket_output, object_name=object_name, file_path=file_path)

print(
    f"Files uploaded to https://{param_s3_server.replace('9000', '9001')}"
    f"/browser"
    f"/{param_s3_bucket_output}"
    f"/{base64.b64encode(f'{user_prefix}/{unique_folder}/'.encode()).decode()}")